# manually understanding the results by calculating manually

In [1]:
import pandas as pd
import numpy as np

In [2]:
# calculating single asset return (or) spread
# Load data
train = pd.read_csv(r'E:\kaagle-hackathons\MITSUI&CO_Commodity_Prediction_Challenge\code\ares\MITSUI-CO_Commodity_Prediction_Challenge\data\raw\train.csv')
target_pairs = pd.read_csv(r'E:\kaagle-hackathons\MITSUI&CO_Commodity_Prediction_Challenge\code\ares\MITSUI-CO_Commodity_Prediction_Challenge\data\raw\target_pairs.csv')

# Example: target_3 = LME_AH_Close - LME_ZS_Close
asset_A = "LME_AH_Close"
asset_B = "LME_ZS_Close"
lag = 1   # predict next-day return diff

# Compute returns
ret_A = (train[asset_A].shift(-lag) - train[asset_A]) / train[asset_A]
ret_B = (train[asset_B].shift(-lag) - train[asset_B]) / train[asset_B]

# Compute spread return
target_3_manual = ret_A - ret_B


In [3]:
target_3_manual

0      -0.009549
1      -0.000646
2      -0.018732
3      -0.022027
4       0.004717
          ...   
1912   -0.003928
1913    0.006456
1914   -0.007942
1915    0.001571
1916         NaN
Length: 1917, dtype: float64

In [4]:
# compare with train_labels.csv
labels = pd.read_csv(r'E:\kaagle-hackathons\MITSUI&CO_Commodity_Prediction_Challenge\code\ares\MITSUI-CO_Commodity_Prediction_Challenge\data\raw\train_labels.csv')

# Compare manual vs provided

np.allclose(target_3_manual[:-lag], labels["target_3"][:-lag], equal_nan=True)
# → should return True


False

note : even target 3 by looking with naked eye some of from 3 values all the values are matching ,
i.e manual calculations is correct

In [5]:
labels

,date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,0,0.005948,-0.002851,-0.004675,-0.000639,NaN,NaN,-0.006729,0.006066,NaN,...,NaN,0.021239,-0.005595,NaN,-0.004628,0.033793,NaN,0.038234,NaN,0.027310
1,1,0.005783,-0.024118,-0.007052,-0.018955,-0.031852,-0.019452,0.003002,-0.006876,-0.002042,...,0.003377,0.021372,-0.001517,0.012846,0.010547,0.030527,-0.000764,0.025021,0.003548,0.020940
2,2,0.001048,0.023836,-0.008934,-0.022060,NaN,NaN,0.037449,0.007658,NaN,...,-0.006712,0.009308,0.001857,-0.012761,-0.002345,0.017529,-0.005394,0.004835,-0.009075,0.001706
3,3,0.001700,-0.024618,0.011943,0.004778,NaN,NaN,-0.012519,-0.016896,NaN,...,NaN,0.036880,-0.015189,NaN,0.008118,0.001079,NaN,-0.015102,NaN,-0.033010
4,4,-0.003272,0.005234,0.006856,0.013312,0.023953,0.010681,-0.011649,0.002019,0.003897,...,NaN,0.004937,NaN,-0.006673,-0.016105,-0.004885,NaN,NaN,0.009514,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,-0.012596,0.009309,0.007947,0.006477,-0.026216,-0.029487,-0.003287,0.011751,-0.027610,...,NaN,NaN,NaN,0.042052,NaN,NaN,NaN,NaN,0.031006,NaN
1913,1913,-0.000736,-0.001292,-0.005137,-0.007961,-0.008494,-0.011547,0.004263,0.005670,-0.004822,...,-0.012992,-0.003349,0.010912,0.006198,-0.009216,-0.003038,-0.026082,-0.008057,-0.002069,-0.141053
1914,1914,-0.002294,0.012898,0.009978,0.001567,0.002596,-0.007373,0.007554,0.002661,0.004083,...,-0.005240,0.004044,0.004459,0.002619,0.001308,-0.006772,-0.019918,-0.013304,-0.005527,-0.127688
1915,1915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.008250,0.012842,0.009076,0.000932,0.011613,0.003825,0.024350,-0.006928,0.006805,-0.012187


calculate pre-computations till given lag column in test-pairs.csv
Eg; if lag = 4 then calculate only for lag1, lag2, lag3, lag4

In [10]:
# ==============================
# 1. Load raw train + labels + pairs
# ==============================
train = train
test_pairs = target_pairs


# ==============================
# 2. Fill missing values in raw train data
# ==============================
train_ffill = train.set_index("date_id").ffill().bfill().reset_index()


# ==============================
# 3. Compute spreads and lag features (optimized)
# ==============================
features = pd.DataFrame()
features["date_id"] = train_ffill["date_id"]

all_spreads = []  # collect DataFrames to concat once at end

for _, row in test_pairs.iterrows():
    target = row["target"]
    lag = int(row["lag"])
    pair = row["pair"]

    # Split pair expression
    # key logic for price - difference, if its single assest , take as it is, else difference
    if "-" in pair:
        left, right = [p.strip() for p in pair.split("-")]
        spread = train_ffill[left] - train_ffill[right]
    else:
        spread = train_ffill[pair]

    # Dict for this target’s features
    feat_dict = {
        f"{target}_spread": spread
    }

    # Add lag features up to given lag
    for l in range(1, lag + 1):
        feat_dict[f"{target}_spread_lag{l}"] = spread.shift(l)

    # Convert dict → DataFrame and append to list
    all_spreads.append(pd.DataFrame(feat_dict, index=train_ffill.index))

# Concatenate all spreads & lags at once with date_id
features = pd.concat([features] + all_spreads, axis=1)

# using ffill at the end, since there are so many nans
features = features.set_index('date_id').ffill().bfill().reset_index()





# # ==============================
# # 4. Merge features with labels
# # ==============================
# # This ensures X and y are aligned on date_id
# train_merged = features.merge(train_labels, on="date_id", how="inner")

# # ==============================
# # 5. Save to parquet (optional for Kaggle dataset upload)
# # ==============================
# train_merged.to_parquet("train_features.parquet", index=False)

# print("✅ Precomputation done! Shape:", train_merged.shape)
# print(train_merged.head())


In [8]:
features

,date_id,target_0_spread,target_0_spread_lag1,target_1_spread,target_1_spread_lag1,target_2_spread,target_2_spread_lag1,target_3_spread,target_3_spread_lag1,target_4_spread,...,target_422_spread,target_422_spread_lag1,target_422_spread_lag2,target_422_spread_lag3,target_422_spread_lag4,target_423_spread,target_423_spread_lag1,target_423_spread_lag2,target_423_spread_lag3,target_423_spread_lag4
0,0,63.9271,NaN,2506.0729,NaN,3856.0,NaN,-1084.5,NaN,-2465.5,...,3426.880370,NaN,NaN,NaN,NaN,7195.5761,NaN,NaN,NaN,NaN
1,1,64.3623,63.9271,2514.6377,2506.0729,3820.0,3856.0,-1099.0,-1084.5,-2502.0,...,3426.879480,3426.880370,NaN,NaN,NaN,7137.5858,7195.5761,NaN,NaN,NaN
2,2,64.7463,64.3623,2522.2537,2514.6377,3826.5,3820.0,-1112.0,-1099.0,-2480.0,...,3426.879191,3426.879480,3426.880370,NaN,NaN,7178.9886,7137.5858,7195.5761,NaN,NaN
3,3,65.1218,64.7463,2474.8782,2522.2537,3767.0,3826.5,-1151.5,-1112.0,-2577.5,...,3485.878979,3426.879191,3426.879480,3426.880370,NaN,7111.4789,7178.9886,7137.5858,7195.5761,NaN
4,4,65.1901,65.1218,2538.8099,2474.8782,3739.0,3767.0,-1211.0,-1151.5,-2605.0,...,3485.878945,3485.878979,3426.879191,3426.879480,3426.880370,7115.7026,7111.4789,7178.9886,7137.5858,7195.5761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,123.9600,123.6200,1837.5400,1876.3800,6847.0,6756.0,-226.5,-210.0,-12690.0,...,4505.919032,4472.919343,4457.919856,4544.918986,4521.919018,9472.3200,9396.2300,9525.4100,9553.2000,9547.5400
1913,1913,123.8300,123.9600,1856.6700,1837.5400,6809.0,6847.0,-239.0,-226.5,-12549.5,...,4553.919329,4505.919032,4472.919343,4457.919856,4544.918986,9467.1000,9472.3200,9396.2300,9525.4100,9553.2000
1914,1914,122.2800,123.8300,1851.7200,1856.6700,6840.5,6809.0,-221.5,-239.0,-12948.5,...,4659.918917,4553.919329,4505.919032,4472.919343,4457.919856,9481.3300,9467.1000,9472.3200,9396.2300,9525.4100
1915,1915,122.1900,122.2800,1847.8100,1851.7200,6803.0,6840.5,-241.5,-221.5,-12998.0,...,4721.918830,4659.918917,4553.919329,4505.919032,4472.919343,9447.6900,9481.3300,9467.1000,9472.3200,9396.2300


In [11]:
features


,date_id,target_0_spread,target_0_spread_lag1,target_1_spread,target_1_spread_lag1,target_2_spread,target_2_spread_lag1,target_3_spread,target_3_spread_lag1,target_4_spread,...,target_422_spread,target_422_spread_lag1,target_422_spread_lag2,target_422_spread_lag3,target_422_spread_lag4,target_423_spread,target_423_spread_lag1,target_423_spread_lag2,target_423_spread_lag3,target_423_spread_lag4
0,0,63.9271,63.9271,2506.0729,2506.0729,3856.0,3856.0,-1084.5,-1084.5,-2465.5,...,3426.880370,3426.880370,3426.880370,3426.880370,3426.880370,7195.5761,7195.5761,7195.5761,7195.5761,7195.5761
1,1,64.3623,63.9271,2514.6377,2506.0729,3820.0,3856.0,-1099.0,-1084.5,-2502.0,...,3426.879480,3426.880370,3426.880370,3426.880370,3426.880370,7137.5858,7195.5761,7195.5761,7195.5761,7195.5761
2,2,64.7463,64.3623,2522.2537,2514.6377,3826.5,3820.0,-1112.0,-1099.0,-2480.0,...,3426.879191,3426.879480,3426.880370,3426.880370,3426.880370,7178.9886,7137.5858,7195.5761,7195.5761,7195.5761
3,3,65.1218,64.7463,2474.8782,2522.2537,3767.0,3826.5,-1151.5,-1112.0,-2577.5,...,3485.878979,3426.879191,3426.879480,3426.880370,3426.880370,7111.4789,7178.9886,7137.5858,7195.5761,7195.5761
4,4,65.1901,65.1218,2538.8099,2474.8782,3739.0,3767.0,-1211.0,-1151.5,-2605.0,...,3485.878945,3485.878979,3426.879191,3426.879480,3426.880370,7115.7026,7111.4789,7178.9886,7137.5858,7195.5761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,123.9600,123.6200,1837.5400,1876.3800,6847.0,6756.0,-226.5,-210.0,-12690.0,...,4505.919032,4472.919343,4457.919856,4544.918986,4521.919018,9472.3200,9396.2300,9525.4100,9553.2000,9547.5400
1913,1913,123.8300,123.9600,1856.6700,1837.5400,6809.0,6847.0,-239.0,-226.5,-12549.5,...,4553.919329,4505.919032,4472.919343,4457.919856,4544.918986,9467.1000,9472.3200,9396.2300,9525.4100,9553.2000
1914,1914,122.2800,123.8300,1851.7200,1856.6700,6840.5,6809.0,-221.5,-239.0,-12948.5,...,4659.918917,4553.919329,4505.919032,4472.919343,4457.919856,9481.3300,9467.1000,9472.3200,9396.2300,9525.4100
1915,1915,122.1900,122.2800,1847.8100,1851.7200,6803.0,6840.5,-241.5,-221.5,-12998.0,...,4721.918830,4659.918917,4553.919329,4505.919032,4472.919343,9447.6900,9481.3300,9467.1000,9472.3200,9396.2300
